In [1]:
print(2)

2


# Final Output Old model

In [20]:
from datetime import datetime
import cv2
import numpy as np
from deepface import DeepFace
import os
import yaml
from yaml.loader import SafeLoader
import time

with open("C:/Users/Gnana Bharathi S/Downloads/dataset/dataset.yaml", mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
print(labels)

# Load YOLO model for gun detection
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/gun_knife_jan_30.onnx')
yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/gun_knife_jan_30.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Gun_Knife20/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load emotion detection model
model = DeepFace.build_model("Emotion")

# Define emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open video capture
#video_path = 'C:/Users/Gnana Bharathi S/Downloads/demo2.mp4'  # Update with your video file path
#video_path='C:/Users/Gnana Bharathi S/Downloads/feb8vid3.mp4'
video_path='C:/Users/Gnana Bharathi S/Downloads/feb8vid1.mp4'
cap = cv2.VideoCapture(video_path)
#cap = cv2.VideoCapture(0)
start_time = None


#yolo_knife = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
yolo_knife = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo_knife.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_knife.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video stream")
        break

    image = frame.copy()

    row, col, d = image.shape  # height, width, depth
    max_rc = max(row, col)
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
    input_image[0:row, 0:col] = image

    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
    yolo.setInput(blob)
    preds = yolo.forward()

    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    image_w, image_h = input_image.shape[:2]
    x_fac = image_w / INPUT_WH_YOLO
    y_fac = image_h / INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * x_fac)
                top = int((cy - 0.5 * h) * y_fac)
                width = int(w * x_fac)
                height = int(h * y_fac)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
    print(result)
    if len(result) > 0:
        flattened_result = result.flatten()
        for ind in flattened_result:
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = labels[classes_id]

            # Draw a rectangle around the gun and label it as 'gun'=='knife'
            if('gun' in class_name):
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)
                cv2.putText(image, 'Gun', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 255, 255), 1)
                #cv2.putText(image, f'Gun {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 255, 255), 1)
    else:
        yolo_knife.setInput(blob)
        preds = yolo_knife.forward()

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []

        image_w, image_h = input_image.shape[:2]
        x_fac = image_w / INPUT_WH_YOLO
        y_fac = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]
            if confidence > 0:
                class_score = row[5:].max()
                class_id = row[5:].argmax()
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    left = int((cx - 0.5 * w) * x_fac)
                    top = int((cy - 0.5 * h) * y_fac)
                    width = int(w * x_fac)
                    height = int(h * y_fac)
                    box = np.array([left, top, width, height])
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        boxes_np = np.array(boxes).tolist()
        confidences_np = np.array(confidences).tolist()

        result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
        print(result)
        if len(result) > 0:
            flattened_result = result.flatten()
            for ind in flattened_result:
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = labels[classes_id]

                # Draw a rectangle around the gun and label it as 'gun'
                #print('bb_conf = ',bb_conf)
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(image, 'Knife', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                #cv2.putText(image, f'Knife {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)

    # Detect emotion of the person in the frame
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray_frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (48, 48), interpolation=cv2.INTER_AREA)
        normalized_face = resized_face / 255.0
        reshaped_face = normalized_face.reshape(1, 48, 48, 1)
        preds = model.predict(reshaped_face)[0]
        emotion_idx = preds.argmax()
        emotion = emotion_labels[emotion_idx]
        print(emotion)
        confidence_percentage = preds[emotion_idx] * 100
        # If gun is detected and emotion is sad or angry, stop running the camera after 3 seconds
        if start_time is None:
            start_time = time.time()  # Store the current time
        else:
            current_time = time.time()  # Get the current time
            elapsed_time = current_time - start_time  # Calculate the elapsed time
            if elapsed_time > 60:  # Check if 30 seconds have passed
                print(emotion)
                cap.release()  # Release the camera
                cv2.destroyAllWindows()  # Close the windows
                break  # Break out of the while loop

        # Draw a rectangle around the emotion and label it with predicted emotion
        color = (0,0,255) if emotion == 'angry' else (0, 0, 255)
        text = emotion
        #text = f'{emotion}: {confidence_percentage:.2f}%'
        if(confidence_percentage<90):
            cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
            cv2.putText(image, text, (x, y+100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Video", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


from datetime import datetime
current_time = datetime.now()
hours = current_time.hour
minutes = current_time.minute
seconds = current_time.second
day=month=current_time.day
month=current_time.month
year=current_time.year

# Print the extracted values
print(f"Current Time: {current_time}")
print(f"Hours: {hours}")
print(f"Minutes: {minutes}")
print(f"Seconds: {seconds}")
print(f"Day: {day}")
print(f"Month: {month}")
print(f"Year: {year}")

result = get_lat_long_by_ip()
latitude, longitude = result
print('latitude = ',latitude)
print('longitude = ',longitude)
x, y = convert_lat_lon_to_xy(latitude, longitude)
print(f"X Coordinate: {x}, Y Coordinate: {y}")

# Replace these values with your own data
#new_data = np.array([[0,0,latitude, longitude, x, y, year, month, day, hours, minutes,seconds]])
new_data = np.array([[1,2,41.830482, -87.621752, 1182280, 1881621, 2023, 12, 15, 14, 30,0]])
#new_data=np.array([[0,1,41.747610,-87.549179,1198234.0,1851595.0,2020,7,1,10,16,0]])
# Standardize the new data using the same scaler
#new_data_standardized = scaler.transform(new_data)

# Make predictions for the new data
new_data_prediction = loaded_model.predict(new_data)

print(f'Predicted Alert for the new data: {new_data_prediction[0]}')

{0: 'Knife', 1: 'Handgun'}
()
[24502]
()
[24502]
()
[24502]
Current Time: 2024-03-20 20:49:24.671503
Hours: 20
Minutes: 49
Seconds: 24
Day: 20
Month: 3
Year: 2024


NameError: name 'get_lat_long_by_ip' is not defined

In [1]:
import requests

def get_lat_long_by_ip():
    try:
        response = requests.get('https://ipinfo.io/json')
        data = response.json()

        if 'loc' in data:
            latitude, longitude = map(float, data['loc'].split(','))
            latitude+=28
            longitude+=7
            longitude*=-1
            return latitude, longitude
        else:
            return None
    except Exception as e:
        print()
        return None

# Example usage:
result = get_lat_long_by_ip()

if result:
    latitude, longitude = result
    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print("Could not retrieve location information.")
import math

def convert_lat_lon_to_xy(latitude, longitude):
    # Radius of the Earth in meters
    R = 6371000.0
    
    # Convert latitude and longitude to radians
    lat_rad = math.radians(latitude)
    lon_rad = math.radians(longitude)
    
    #y Calculate x and y coordinates using Mercator projection
    x = R * lon_rad
    y = R * math.log(math.tan(math.pi / 4 + lat_rad / 2))
    
    return x, y
x,y=convert_lat_lon_to_xy(41.0878,-87.2785)
x*=-1
x-=8564867
y-=3164867
print(x,y)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

import joblib
loaded_model = joblib.load("C:/Users/Gnana Bharathi S/Downloads/Dec 14/RF1_81.pkl")
new_data=np.array([[0,3,41.840482,-87.621752,1178180.0,1881621.0,2020,10,20,12,23,0]])
# Standardize the new data using the same scaler
#new_data_standardized = scaler.transform(new_data)

# Make predictions for the new data
new_data_prediction = loaded_model.predict(new_data)
new_data_prediction[0]

Latitude: 41.0878, Longitude: -87.2785
1140059.4051471204 1854810.5772076007


C:\Users\Gnana Bharathi S\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'Safe'

# Final Output new model

In [2]:
from datetime import datetime
import cv2
import numpy as np
from deepface import DeepFace
import os
import yaml
from yaml.loader import SafeLoader
import time

with open("C:/Users/Gnana Bharathi S/Downloads/Gun_Knife/weights/names.yaml", mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
print(labels)
# Load YOLO model for gun detection
yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Weapon Model/best.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Gun_Knifea03/weights/best.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load emotion detection model
model = DeepFace.build_model("Emotion")

# Define emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open video capture
#video_path = 'C:/Users/Gnana Bharathi S/Downloads/demo2.mp4'  # Update with your video file path
#video_path='C:/Users/Gnana Bharathi S/Downloads/feb8vid3.mp4'
video_path='C:/Users/Gnana Bharathi S/Downloads/feb8vid.mp4'
#cap = cv2.VideoCapture(video_path)
cap = cv2.VideoCapture(0)
start_time = None


'''yolo_knife = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo_knife.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_knife.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
'''
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video stream")
        break

    image = frame.copy()

    row, col, d = image.shape  # height, width, depth
    max_rc = max(row, col)
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
    input_image[0:row, 0:col] = image

    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
    yolo.setInput(blob)
    preds = yolo.forward()

    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    image_w, image_h = input_image.shape[:2]
    x_fac = image_w / INPUT_WH_YOLO
    y_fac = image_h / INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * x_fac)
                top = int((cy - 0.5 * h) * y_fac)
                width = int(w * x_fac)
                height = int(h * y_fac)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
    print(result)
    if len(result) > 0:
        flattened_result = result.flatten()
        for ind in flattened_result:
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = labels[classes_id]
            if('gun' in class_name):
                wep=0
            elif('knife' in class_name):
                wep=1
            else:
                wep=2
            #text=class_name+' = '+str(bb_conf)+'%'
            text=class_name
            # Draw a rectangle around the gun and label it as 'gun'=='knife'
            if('gun' in class_name):
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(image, text, (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
            elif('knife' in class_name):
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)
                cv2.putText(image, text, (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 255, 255), 1)
                
                #cv2.putText(image, f'Gun {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 255, 255), 1)
    '''else:
        yolo_knife.setInput(blob)
        preds = yolo_knife.forward()

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []

        image_w, image_h = input_image.shape[:2]
        x_fac = image_w / INPUT_WH_YOLO
        y_fac = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]
            if confidence > 0:
                class_score = row[5:].max()
                class_id = row[5:].argmax()
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    left = int((cx - 0.5 * w) * x_fac)
                    top = int((cy - 0.5 * h) * y_fac)
                    width = int(w * x_fac)
                    height = int(h * y_fac)
                    box = np.array([left, top, width, height])
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        boxes_np = np.array(boxes).tolist()
        confidences_np = np.array(confidences).tolist()

        result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
        print(result)
        if len(result) > 0:
            flattened_result = result.flatten()
            for ind in flattened_result:
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = labels[classes_id]

                # Draw a rectangle around the gun and label it as 'gun'
                #print('bb_conf = ',bb_conf)
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(image, 'Knife', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                #cv2.putText(image, f'Knife {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)'''      

    # Detect emotion of the person in the frame
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray_frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (48, 48), interpolation=cv2.INTER_AREA)
        normalized_face = resized_face / 255.0
        reshaped_face = normalized_face.reshape(1, 48, 48, 1)
        preds = model.predict(reshaped_face)[0]
        emotion_idx = preds.argmax()
        emotion = emotion_labels[emotion_idx]
        print(emotion)
        confidence_percentage = preds[emotion_idx] * 100
        # If gun is detected and emotion is sad or angry, stop running the camera after 3 seconds
        if start_time is None:
            start_time = time.time()  # Store the current time
        else:
            current_time = time.time()  # Get the current time
            elapsed_time = current_time - start_time  # Calculate the elapsed time
            if elapsed_time > 1000:  # Check if 30 seconds have passed
                print(emotion)
                cap.release()  # Release the camera
                cv2.destroyAllWindows()  # Close the windows
                break  # Break out of the while loop

        # Draw a rectangle around the emotion and label it with predicted emotion
        color = (0,0,255) if emotion == 'angry' else (0, 0, 255)
        text = emotion
        #text = f'{emotion}: {confidence_percentage:.2f}%'
        if(confidence_percentage<90):
            cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
            cv2.putText(image, text, (x, y+100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Video", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

from datetime import datetime
current_time = datetime.now()
hours = current_time.hour
minutes = current_time.minute
seconds = current_time.second
day=month=current_time.day
month=current_time.month
year=current_time.year

# Print the extracted values
print(f"Current Time: {current_time}")
print(f"Hours: {hours}")
print(f"Minutes: {minutes}")
print(f"Seconds: {seconds}")
print(f"Day: {day}")
print(f"Month: {month}")
print(f"Year: {year}")

result = get_lat_long_by_ip()
latitude, longitude = result
print('latitude = ',latitude)
print('longitude = ',longitude)
x, y = convert_lat_lon_to_xy(latitude, longitude)
print(f"X Coordinate: {x}, Y Coordinate: {y}")
em=0
if(emotion=='angry'):
    em=0
elif(emotion=='disgust'):
    em=1
elif(emotion=='fear'):
    em=2
elif(emotion=='happy'):
    em=3
elif(emotion=='neutral'):
    em=4
elif(emotion=='sad'):
    em=5
elif(emotion=='surprise'):
    em=6
else:
    em=4
print(wep)
print(em)

# Replace these values with your own data
new_data = np.array([[wep,em,latitude, longitude, x, y, year, month, day, hours, minutes,seconds]])
#new_data = np.array([[1,2,41.830482, -87.621752, 1182280, 1881621, 2023, 12, 15, 14, 30,0]])
#new_data=np.array([[0,1,41.747610,-87.549179,1198234.0,1851595.0,2020,7,1,10,16,0]])
# Standardize the new data using the same scaler
#new_data_standardized = scaler.transform(new_data)

# Make predictions for the new data
new_data_prediction = loaded_model.predict(new_data)

print(f'Predicted Alert for the new data: {new_data_prediction[0]}')



['gun', 'knife']
()
[24990]
()
()
()
()
[24128]
1/1 [==============================] - 0s 432ms/step
sad
()
()
1/1 [==============================] - 0s 73ms/step
neutral
()
[22697]
[24148]
[24168]
1/1 [==============================] - 0s 120ms/step
fear
[23179]
()
1/1 [==============================] - 0s 53ms/step
fear
1/1 [==============================] - 0s 34ms/step
sad
[23132]
[23134]
1/1 [==============================] - 0s 31ms/step
sad
[24148]
()
()
[19406 14842]
1/1 [==============================] - 0s 49ms/step
sad
[19529 19781]
[19569  2442]
()
()
[19933]
1/1 [==============================] - 0s 31ms/step
fear
[19893]
1/1 [==============================] - 0s 32ms/step
fear
[21492]
1/1 [==============================] - 0s 31ms/step
fear
[24165]
1/1 [==============================] - 0s 31ms/step
neutral
Current Time: 2024-04-16 21:42:07.461367
Hours: 21
Minutes: 42
Seconds: 7
Day: 16
Month: 4
Year: 2024
latitude =  41.0878
longitude =  -87.2785
X Coordinate: -9704926

# Gun, Knife, Police

In [6]:
from datetime import datetime
import cv2
import numpy as np
from deepface import DeepFace
import os
import yaml
from yaml.loader import SafeLoader
import time

with open("C:/Users/Gnana Bharathi S/Downloads/Gun_Knife/weights/names.yaml", mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
print(labels)



with open("C:/Users/Gnana Bharathi S/Downloads/Feb23_Police/data_1.yaml", mode='r') as f:
    pol_yaml = yaml.load(f, Loader=SafeLoader)

labels1 = pol_yaml['names']
print(labels1)

# Load YOLO model for gun detection
yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Weapon Model/best.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load emotion detection model
model = DeepFace.build_model("Emotion")

# Define emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open video capture
#video_path = 'C:/Users/Gnana Bharathi S/Downloads/demo2.mp4'  # Update with your video file path
#video_path='C:/Users/Gnana Bharathi S/Downloads/feb8vid3.mp4'
video_path='C:/Users/Gnana Bharathi S/Downloads/pol4.mp4'
#video_path="C:/Users/Gnana Bharathi S/Videos/20240313_110450.mp4"
cap = cv2.VideoCapture(video_path)
#cap = cv2.VideoCapture(0)
start_time = None


'''yolo_knife = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo_knife.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_knife.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)'''
frame_skip = 5  # Process every 5th frame
frame_count = 0

#Police
yolo_p = cv2.dnn.readNetFromONNX("C:/Users/Gnana Bharathi S/Downloads/Feb23_Police/Feb23_Police.onnx")
yolo_p.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_p.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

gun=2
knife=0
em=0
emotion=''
wep=2
pol=0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video stream")
        break
    #frame_count += 1
    #if frame_count % frame_skip != 0:
     #   continue

    image = frame.copy()
    resize_width = 480
    resize_height = 480
    input_image = cv2.resize(image, (resize_width, resize_height))
    row, col, d = image.shape  # height, width, depth
    max_rc = max(row, col)
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
    
    input_image[0:row, 0:col] = image
    
    
    
    
    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1 / 255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
    yolo_p.setInput(blob)
    preds = yolo_p.forward()

    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    image_w, image_h = input_image.shape[:2]
    x_fac = image_w / INPUT_WH_YOLO
    y_fac = image_h / INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * x_fac)
                top = int((cy - 0.5 * h) * y_fac)
                width = int(w * x_fac)
                height = int(h * y_fac)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)

    if len(result) > 0:
        flattened_result = result.flatten()
        for ind in flattened_result:
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = labels1[classes_id]
            text = f'{class_name}:{bb_conf}%'
            if(('officer' in class_name.lower() or 'text' in class_name.lower()) and bb_conf>70):
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, text, (x, y+20), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 255, 255), 1)
                print('police = ',text)
                pol=1
            #else:
             #   pol=0
            #if('officer' in class_name.lower() or 'text' in class_name.lower()):
             #   pol=1

    
    
    
    
    
    
    
    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
    yolo.setInput(blob)
    preds = yolo.forward()

    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    image_w, image_h = input_image.shape[:2]
    x_fac = image_w / INPUT_WH_YOLO
    y_fac = image_h / INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * x_fac)
                top = int((cy - 0.5 * h) * y_fac)
                width = int(w * x_fac)
                height = int(h * y_fac)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
    print(result)
    if len(result) > 0:
        flattened_result = result.flatten()
        gun=1
        wep=0
        for ind in flattened_result:
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = labels[classes_id]
            print(class_name)
            if('gun' in class_name or 'handgun' in class_name):
            # Draw a rectangle around the gun and label it as 'gun'
                print(bb_conf)
                if(bb_conf>0):
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                    cv2.putText(image, 'Gun', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                    #cv2.putText(image, f'Gun {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
            elif('knife' in class_name):
                if(bb_conf>0):
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                    cv2.putText(image, 'Knife', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                    #cv2.putText(image, f'Gun {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                
    '''else:
        yolo_knife.setInput(blob)
        preds = yolo_knife.forward()

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []

        image_w, image_h = input_image.shape[:2]
        x_fac = image_w / INPUT_WH_YOLO
        y_fac = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]
            if confidence > 0:
                class_score = row[5:].max()
                class_id = row[5:].argmax()
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    left = int((cx - 0.5 * w) * x_fac)
                    top = int((cy - 0.5 * h) * y_fac)
                    width = int(w * x_fac)
                    height = int(h * y_fac)
                    box = np.array([left, top, width, height])
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        boxes_np = np.array(boxes).tolist()
        confidences_np = np.array(confidences).tolist()

        result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
        print(result)
        if len(result) > 0:
            flattened_result = result.flatten()
            knife=1
            wep=1
            for ind in flattened_result:
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = labels[classes_id]

                # Draw a rectangle around the gun and label it as 'gun'
                print('bb_conf = ',bb_conf)
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(image, 'Knife', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                #cv2.putText(image, f'Knife {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)'''

    # Detect emotion of the person in the frame
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray_frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (48, 48), interpolation=cv2.INTER_AREA)
        normalized_face = resized_face / 255.0
        reshaped_face = normalized_face.reshape(1, 48, 48, 1)
        preds = model.predict(reshaped_face)[0]
        emotion_idx = preds.argmax()
        emotion = emotion_labels[emotion_idx]
        print(emotion)
        confidence_percentage = preds[emotion_idx] * 100
        # If gun is detected and emotion is sad or angry, stop running the camera after 3 seconds
        if start_time is None:
            start_time = time.time()  # Store the current time
        else:
            current_time = time.time()  # Get the current time
            elapsed_time = current_time - start_time  # Calculate the elapsed time
            if elapsed_time > 1000:  # Check if 5 seconds have passed
                print(emotion)
                cap.release()  # Release the camera
                cv2.destroyAllWindows()  # Close the windows
                break  # Break out of the while loop

        # Draw a rectangle around the emotion and label it with predicted emotion
        color = (0,0,255) if emotion == 'angry' else (0, 0, 255)
        text = emotion
        #text = f'{emotion}: {confidence_percentage:.2f}%'
        if(confidence_percentage<80):
            cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
            cv2.putText(image, text, (x, y+100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Video", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
if(pol==0):
    from datetime import datetime
    current_time = datetime.now()
    hours = current_time.hour
    minutes = current_time.minute
    seconds = current_time.second
    day=month=current_time.day
    month=current_time.month
    year=current_time.year

    # Print the extracted values
    #print(f"Current Time: {current_time}")
    #print(f"Hours: {hours}")
    #print(f"Minutes: {minutes}")
    #print(f"Seconds: {seconds}")
    #print(f"Day: {day}")
    #print(f"Month: {month}")
    #print(f"Year: {year}")

    result = get_lat_long_by_ip()
    latitude, longitude = result
    #latitude = 41
    #longitude=-87
    #print('latitude = ',latitude)
    #print('longitude = ',longitude)
    x, y = convert_lat_lon_to_xy(latitude, longitude)
    #x=1182280
    #y=1881621
    #print(f"X Coordinate: {x}, Y Coordinate: {y}")

    if(emotion=='angry'):
        em=0
    elif(emotion=='disgust'):
        em=1
    elif(emotion=='fear'):
        em=2
    elif(emotion=='happy'):
        em=3
    elif(emotion=='neutral'):
        em=4
    elif(emotion=='sad'):
        em=5
    elif(emotion=='surprise'):
        em=6
    else:
        em=4
    # Replace these values with your own data
    new_data = np.array([[wep,em,latitude, longitude, x, y, year, month, day, hours, minutes,seconds]])
    #new_data = np.array([[1,2,41.830482, -87.621752, 1182280, 1881621, 2023, 12, 15, 14, 30,0]])
    #new_data=np.array([[0,1,41.747610,-87.549179,1198234.0,1851595.0,2020,7,1,10,16,0]])
    # Standardize the new data using the same scaler
    #new_data_standardized = scaler.transform(new_data)

    # Make predictions for the new data
    new_data_prediction = loaded_model.predict(new_data)

    print(f'Predicted Alert for the new data: {new_data_prediction[0]}')
else:
    print('Safe since police is present')

['gun', 'knife']
['badge', 'baton', 'duty belt', 'handcuffs', 'lights', 'nameplate', 'officer', 'patch', 'police hat', 'radio', 'sunglasses', 'text', 'vehicle', 'watch']
police =  officer:87%
police =  officer:81%
()
police =  officer:87%
police =  officer:80%
()
police =  officer:87%
police =  officer:80%
()
police =  officer:87%
police =  officer:80%
()
Safe since police is present


In [1]:
from datetime import datetime
import cv2
import numpy as np
from deepface import DeepFace
import os
import yaml
from yaml.loader import SafeLoader
import time

In [8]:
import onnx

# Load the YOLO model in ONNX format
model = onnx.load('C:/Users/Gnana Bharathi S/Downloads/Feb23_Police/Feb23_Police.onnx')

# Print model graph to visualize the architecture
print(onnx.helper.printable_graph(model.graph))

graph main_graph (
  %images[FLOAT, 1x3x640x640]
) initializers (
  %model.0.conv.weight[FLOAT, 32x3x6x6]
  %model.0.conv.bias[FLOAT, 32]
  %model.1.conv.weight[FLOAT, 64x32x3x3]
  %model.1.conv.bias[FLOAT, 64]
  %model.2.cv1.conv.weight[FLOAT, 32x64x1x1]
  %model.2.cv1.conv.bias[FLOAT, 32]
  %model.2.cv2.conv.weight[FLOAT, 32x64x1x1]
  %model.2.cv2.conv.bias[FLOAT, 32]
  %model.2.cv3.conv.weight[FLOAT, 64x64x1x1]
  %model.2.cv3.conv.bias[FLOAT, 64]
  %model.2.m.0.cv1.conv.weight[FLOAT, 32x32x1x1]
  %model.2.m.0.cv1.conv.bias[FLOAT, 32]
  %model.2.m.0.cv2.conv.weight[FLOAT, 32x32x3x3]
  %model.2.m.0.cv2.conv.bias[FLOAT, 32]
  %model.3.conv.weight[FLOAT, 128x64x3x3]
  %model.3.conv.bias[FLOAT, 128]
  %model.4.cv1.conv.weight[FLOAT, 64x128x1x1]
  %model.4.cv1.conv.bias[FLOAT, 64]
  %model.4.cv2.conv.weight[FLOAT, 64x128x1x1]
  %model.4.cv2.conv.bias[FLOAT, 64]
  %model.4.cv3.conv.weight[FLOAT, 128x128x1x1]
  %model.4.cv3.conv.bias[FLOAT, 128]
  %model.4.m.0.cv1.conv.weight[FLOAT, 64x64x1

In [15]:
pip install netron

Note: you may need to restart the kernel to use updated packages.


In [17]:
from datetime import datetime
import cv2
import numpy as np
from deepface import DeepFace
import os
import yaml
from yaml.loader import SafeLoader
import time

with open("C:/Users/Gnana Bharathi S/Downloads/dataset/dataset.yaml", mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
print(labels)



with open("C:/Users/Gnana Bharathi S/Downloads/Feb23_Police/data_1.yaml", mode='r') as f:
    pol_yaml = yaml.load(f, Loader=SafeLoader)

labels1 = pol_yaml['names']
print(labels1)

# Load YOLO model for gun detection
yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/C:/Users/Gnana Bharathi S/Downloads/Weapon Model/best.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load emotion detection model
model = DeepFace.build_model("Emotion")

# Define emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open video capture
#video_path = 'C:/Users/Gnana Bharathi S/Downloads/demo2.mp4'  # Update with your video file path
#video_path='C:/Users/Gnana Bharathi S/Downloads/feb8vid1.mp4'
video_path='C:/Users/Gnana Bharathi S/Downloads/pol3.mp4'
#video_path="C:/Users/Gnana Bharathi S/Videos/20240313_110450.mp4"
cap = cv2.VideoCapture(video_path)
#cap = cv2.VideoCapture(0)
start_time = None


'''yolo_knife = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo_knife.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_knife.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)'''

frame_skip = 5  # Process every 5th frame
frame_count = 0

#Police
yolo_p = cv2.dnn.readNetFromONNX("C:/Users/Gnana Bharathi S/Downloads/Feb23_Police/Feb23_Police.onnx")
yolo_p.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_p.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

gun=2
knife=0
em=0
emotion=''
wep=2
pol=0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video stream")
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

{0: 'Knife', 1: 'Handgun'}
['badge', 'baton', 'duty belt', 'handcuffs', 'lights', 'nameplate', 'officer', 'patch', 'police hat', 'radio', 'sunglasses', 'text', 'vehicle', 'watch']
End of video stream


# Gun, Knife, Police

In [7]:
from datetime import datetime
import cv2
import numpy as np
from deepface import DeepFace
import os
import yaml
from yaml.loader import SafeLoader
import time

with open("C:/Users/Gnana Bharathi S/Downloads/Gun_Knife/weights/names.yaml", mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
print(labels)



with open("C:/Users/Gnana Bharathi S/Downloads/Feb23_Police/data_1.yaml", mode='r') as f:
    pol_yaml = yaml.load(f, Loader=SafeLoader)

labels1 = pol_yaml['names']
print(labels1)

# Load YOLO model for gun detection
yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Weapon Model/best.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load emotion detection model
model = DeepFace.build_model("Emotion")

# Define emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open video capture
#video_path = 'C:/Users/Gnana Bharathi S/Downloads/demo2.mp4'  # Update with your video file path
#video_path='C:/Users/Gnana Bharathi S/Downloads/feb8vid3.mp4'
video_path='C:/Users/Gnana Bharathi S/Downloads/pol4.mp4'
#video_path="C:/Users/Gnana Bharathi S/Videos/20240313_110450.mp4"
cap = cv2.VideoCapture(video_path)
#cap = cv2.VideoCapture(0)
start_time = None


'''yolo_knife = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo_knife.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_knife.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)'''
frame_skip = 5  # Process every 5th frame
frame_count = 0

#Police
yolo_p = cv2.dnn.readNetFromONNX("C:/Users/Gnana Bharathi S/Downloads/Feb23_Police/Feb23_Police.onnx")
yolo_p.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_p.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

gun=2
knife=0
em=0
emotion=''
wep=2
pol=0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video stream")
        break
    #frame_count += 1
    #if frame_count % frame_skip != 0:
     #   continue

    image = frame.copy()
    resize_width = 480
    resize_height = 480
    input_image = cv2.resize(image, (resize_width, resize_height))
    row, col, d = image.shape  # height, width, depth
    max_rc = max(row, col)
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
    
    input_image[0:row, 0:col] = image
    
    
    
    
    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1 / 255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
    yolo_p.setInput(blob)
    preds = yolo_p.forward()

    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    image_w, image_h = input_image.shape[:2]
    x_fac = image_w / INPUT_WH_YOLO
    y_fac = image_h / INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * x_fac)
                top = int((cy - 0.5 * h) * y_fac)
                width = int(w * x_fac)
                height = int(h * y_fac)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)

    if len(result) > 0:
        flattened_result = result.flatten()
        for ind in flattened_result:
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = labels1[classes_id]
            text = f'{class_name}:{bb_conf}%'
            if(('officer' in class_name.lower() or 'text' in class_name.lower()) and bb_conf>70):
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, text, (x, y+20), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 255, 255), 1)
                print('police = ',text)
                pol=1
            #else:
             #   pol=0
            #if('officer' in class_name.lower() or 'text' in class_name.lower()):
             #   pol=1

    
    
    
    
    
    
    
    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
    yolo.setInput(blob)
    preds = yolo.forward()

    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    image_w, image_h = input_image.shape[:2]
    x_fac = image_w / INPUT_WH_YOLO
    y_fac = image_h / INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * x_fac)
                top = int((cy - 0.5 * h) * y_fac)
                width = int(w * x_fac)
                height = int(h * y_fac)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
    print(result)
    if len(result) > 0:
        flattened_result = result.flatten()
        gun=1
        wep=0
        for ind in flattened_result:
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = labels[classes_id]
            print(class_name)
            if('gun' in class_name or 'handgun' in class_name):
            # Draw a rectangle around the gun and label it as 'gun'
                print(bb_conf)
                if(bb_conf>0):
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                    cv2.putText(image, 'Gun', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                    #cv2.putText(image, f'Gun {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
            elif('knife' in class_name):
                if(bb_conf>0):
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                    cv2.putText(image, 'Knife', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                    #cv2.putText(image, f'Gun {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                
    '''else:
        yolo_knife.setInput(blob)
        preds = yolo_knife.forward()

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []

        image_w, image_h = input_image.shape[:2]
        x_fac = image_w / INPUT_WH_YOLO
        y_fac = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]
            if confidence > 0:
                class_score = row[5:].max()
                class_id = row[5:].argmax()
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    left = int((cx - 0.5 * w) * x_fac)
                    top = int((cy - 0.5 * h) * y_fac)
                    width = int(w * x_fac)
                    height = int(h * y_fac)
                    box = np.array([left, top, width, height])
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        boxes_np = np.array(boxes).tolist()
        confidences_np = np.array(confidences).tolist()

        result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
        print(result)
        if len(result) > 0:
            flattened_result = result.flatten()
            knife=1
            wep=1
            for ind in flattened_result:
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = labels[classes_id]

                # Draw a rectangle around the gun and label it as 'gun'
                print('bb_conf = ',bb_conf)
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(image, 'Knife', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                #cv2.putText(image, f'Knife {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)'''

    # Detect emotion of the person in the frame
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray_frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (48, 48), interpolation=cv2.INTER_AREA)
        normalized_face = resized_face / 255.0
        reshaped_face = normalized_face.reshape(1, 48, 48, 1)
        preds = model.predict(reshaped_face)[0]
        emotion_idx = preds.argmax()
        emotion = emotion_labels[emotion_idx]
        print(emotion)
        confidence_percentage = preds[emotion_idx] * 100
        # If gun is detected and emotion is sad or angry, stop running the camera after 3 seconds
        if start_time is None:
            start_time = time.time()  # Store the current time
        else:
            current_time = time.time()  # Get the current time
            elapsed_time = current_time - start_time  # Calculate the elapsed time
            if elapsed_time > 1000:  # Check if 5 seconds have passed
                print(emotion)
                cap.release()  # Release the camera
                cv2.destroyAllWindows()  # Close the windows
                break  # Break out of the while loop

        # Draw a rectangle around the emotion and label it with predicted emotion
        color = (0,0,255) if emotion == 'angry' else (0, 0, 255)
        text = emotion
        #text = f'{emotion}: {confidence_percentage:.2f}%'
        if(confidence_percentage<80):
            cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
            cv2.putText(image, text, (x, y+100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Video", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
if(pol==0):
    from datetime import datetime
    current_time = datetime.now()
    hours = current_time.hour
    minutes = current_time.minute
    seconds = current_time.second
    day=month=current_time.day
    month=current_time.month
    year=current_time.year

    # Print the extracted values
    #print(f"Current Time: {current_time}")
    #print(f"Hours: {hours}")
    #print(f"Minutes: {minutes}")
    #print(f"Seconds: {seconds}")
    #print(f"Day: {day}")
    #print(f"Month: {month}")
    #print(f"Year: {year}")

    result = get_lat_long_by_ip()
    latitude, longitude = result
    #latitude = 41
    #longitude=-87
    #print('latitude = ',latitude)
    #print('longitude = ',longitude)
    x, y = convert_lat_lon_to_xy(latitude, longitude)
    #x=1182280
    #y=1881621
    #print(f"X Coordinate: {x}, Y Coordinate: {y}")

    if(emotion=='angry'):
        em=0
    elif(emotion=='disgust'):
        em=1
    elif(emotion=='fear'):
        em=2
    elif(emotion=='happy'):
        em=3
    elif(emotion=='neutral'):
        em=4
    elif(emotion=='sad'):
        em=5
    elif(emotion=='surprise'):
        em=6
    else:
        em=4
    # Replace these values with your own data
    new_data = np.array([[wep,em,latitude, longitude, x, y, year, month, day, hours, minutes,seconds]])
    #new_data = np.array([[1,2,41.830482, -87.621752, 1182280, 1881621, 2023, 12, 15, 14, 30,0]])
    #new_data=np.array([[0,1,41.747610,-87.549179,1198234.0,1851595.0,2020,7,1,10,16,0]])
    # Standardize the new data using the same scaler
    #new_data_standardized = scaler.transform(new_data)

    # Make predictions for the new data
    new_data_prediction = loaded_model.predict(new_data)

    print(f'Predicted Alert for the new data: {new_data_prediction[0]}')
else:
    print('Safe since police is present')

['gun', 'knife']
['badge', 'baton', 'duty belt', 'handcuffs', 'lights', 'nameplate', 'officer', 'patch', 'police hat', 'radio', 'sunglasses', 'text', 'vehicle', 'watch']
police =  officer:87%
police =  officer:81%
()
police =  officer:87%
police =  officer:80%
()
police =  officer:87%
police =  officer:80%
()
Safe since police is present


# Final Code

In [27]:
from datetime import datetime
import cv2
import numpy as np
from deepface import DeepFace
import os
import yaml
from yaml.loader import SafeLoader
import time

with open("C:/Users/Gnana Bharathi S/Downloads/Gun_Knife/weights/names.yaml", mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
print(labels)
# Load YOLO model for gun detection
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/gun_knife_jan_30.onnx')
yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Gun_Knifea03/weights/best.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Load emotion detection model
model = DeepFace.build_model("Emotion")

# Define emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open video capture
#video_path = 'C:/Users/Gnana Bharathi S/Downloads/demo2.mp4'  # Update with your video file path
video_path='C:/Users/Gnana Bharathi S/Downloads/pol3.mp4'
video_path='C:/Users/Gnana Bharathi S/Downloads/feb8vid.mp4'
#cap = cv2.VideoCapture(video_path)
cap = cv2.VideoCapture(0)
start_time = None

yolo_p = cv2.dnn.readNetFromONNX("C:/Users/Gnana Bharathi S/Downloads/Feb23_Police/Feb23_Police.onnx")
yolo_p.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_p.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)


'''yolo_knife = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/Dec 14/knife.onnx')
#yolo = cv2.dnn.readNetFromONNX('C:/Users/Gnana Bharathi S/Downloads/feb3.onnx')
yolo_knife.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_knife.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
'''
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video stream")
        break

    image = frame.copy()

    row, col, d = image.shape  # height, width, depth
    max_rc = max(row, col)
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
    input_image[0:row, 0:col] = image
    
    
    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1 / 255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
    yolo_p.setInput(blob)
    preds = yolo_p.forward()

    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    image_w, image_h = input_image.shape[:2]
    x_fac = image_w / INPUT_WH_YOLO
    y_fac = image_h / INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * x_fac)
                top = int((cy - 0.5 * h) * y_fac)
                width = int(w * x_fac)
                height = int(h * y_fac)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)

    if len(result) > 0:
        flattened_result = result.flatten()
        for ind in flattened_result:
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = labels1[classes_id]
            text = f'{class_name}:{bb_conf}%'
            if(('officer' in class_name.lower() or 'text' in class_name.lower()) and bb_conf>65):
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, text, (x, y+20), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 255, 255), 1)
                print('police = ',text)
                pol=1
    
    
    

    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
    yolo.setInput(blob)
    preds = yolo.forward()

    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    image_w, image_h = input_image.shape[:2]
    x_fac = image_w / INPUT_WH_YOLO
    y_fac = image_h / INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]
        if confidence > 0:
            class_score = row[5:].max()
            class_id = row[5:].argmax()
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * x_fac)
                top = int((cy - 0.5 * h) * y_fac)
                width = int(w * x_fac)
                height = int(h * y_fac)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
    print(result)
    if len(result) > 0:
        flattened_result = result.flatten()
        for ind in flattened_result:
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = labels[classes_id]
            text=class_name+' = '+str(bb_conf)+'%'
            # Draw a rectangle around the gun and label it as 'gun'=='knife'
            #if('gun' in class_name):
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)
            cv2.putText(image, text, (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 255, 255), 1)
                #cv2.putText(image, f'Gun {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 255, 255), 1)
    '''else:
        yolo_knife.setInput(blob)
        preds = yolo_knife.forward()

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []

        image_w, image_h = input_image.shape[:2]
        x_fac = image_w / INPUT_WH_YOLO
        y_fac = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]
            if confidence > 0:
                class_score = row[5:].max()
                class_id = row[5:].argmax()
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    left = int((cx - 0.5 * w) * x_fac)
                    top = int((cy - 0.5 * h) * y_fac)
                    width = int(w * x_fac)
                    height = int(h * y_fac)
                    box = np.array([left, top, width, height])
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        boxes_np = np.array(boxes).tolist()
        confidences_np = np.array(confidences).tolist()

        result = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.3, 0.2)
        print(result)
        if len(result) > 0:
            flattened_result = result.flatten()
            for ind in flattened_result:
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = labels[classes_id]

                # Draw a rectangle around the gun and label it as 'gun'
                #print('bb_conf = ',bb_conf)
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(image, 'Knife', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
                #cv2.putText(image, f'Knife {bb_conf}% confidence', (x, y+10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)'''      

    # Detect emotion of the person in the frame
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray_frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (48, 48), interpolation=cv2.INTER_AREA)
        normalized_face = resized_face / 255.0
        reshaped_face = normalized_face.reshape(1, 48, 48, 1)
        preds = model.predict(reshaped_face)[0]
        emotion_idx = preds.argmax()
        emotion = emotion_labels[emotion_idx]
        print(emotion)
        confidence_percentage = preds[emotion_idx] * 100
        # If gun is detected and emotion is sad or angry, stop running the camera after 3 seconds
        if start_time is None:
            start_time = time.time()  # Store the current time
        else:
            current_time = time.time()  # Get the current time
            elapsed_time = current_time - start_time  # Calculate the elapsed time
            if elapsed_time > 60:  # Check if 30 seconds have passed
                print(emotion)
                cap.release()  # Release the camera
                cv2.destroyAllWindows()  # Close the windows
                break  # Break out of the while loop

        # Draw a rectangle around the emotion and label it with predicted emotion
        color = (0,0,255) if emotion == 'angry' else (0, 0, 255)
        text = emotion
        #text = f'{emotion}: {confidence_percentage:.2f}%'
        if(confidence_percentage<90):
            cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
            cv2.putText(image, text, (x, y+100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Video", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

from datetime import datetime
current_time = datetime.now()
hours = current_time.hour
minutes = current_time.minute
seconds = current_time.second
day=month=current_time.day
month=current_time.month
year=current_time.year

# Print the extracted values
print(f"Current Time: {current_time}")
print(f"Hours: {hours}")
print(f"Minutes: {minutes}")
print(f"Seconds: {seconds}")
print(f"Day: {day}")
print(f"Month: {month}")
print(f"Year: {year}")

result = get_lat_long_by_ip()
latitude, longitude = result
print('latitude = ',latitude)
print('longitude = ',longitude)
x, y = convert_lat_lon_to_xy(latitude, longitude)
print(f"X Coordinate: {x}, Y Coordinate: {y}")

# Replace these values with your own data
#new_data = np.array([[0,0,latitude, longitude, x, y, year, month, day, hours, minutes,seconds]])
new_data = np.array([[1,2,41.830482, -87.621752, 1182280, 1881621, 2023, 12, 15, 14, 30,0]])
#new_data=np.array([[0,1,41.747610,-87.549179,1198234.0,1851595.0,2020,7,1,10,16,0]])
# Standardize the new data using the same scaler
#new_data_standardized = scaler.transform(new_data)

# Make predictions for the new data
new_data_prediction = loaded_model.predict(new_data)

print(f'Predicted Alert for the new data: {new_data_prediction[0]}')


['gun', 'knife']
()
1/1 [==============================] - 0s 44ms/step
neutral
()
[24722]
[24493 24314]
()
()
()
()
()
()
()
()
1/1 [==============================] - 0s 31ms/step
angry
()
()
[24336]
1/1 [==============================] - 0s 56ms/step
angry
()
()
()
()
()
[25083]
[24076 24184]
[24211]
Current Time: 2024-04-04 22:22:05.178202
Hours: 22
Minutes: 22
Seconds: 5
Day: 4
Month: 4
Year: 2024
latitude =  41.0878
longitude =  -87.2785
X Coordinate: -9704926.40514712, Y Coordinate: 5019677.577207601
Predicted Alert for the new data: Danger


In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Mapping between Original and Encoded Values:
   Original Encoded
0     angry       0
1   disgust       1
2      fear       2
3     happy       3
4   neutral       4
5       sad       5
6  surprise       6

0     gun       0
1   knife       1

In [3]:
import requests

def get_lat_long_by_ip():
    try:
        response = requests.get('https://ipinfo.io/json')
        data = response.json()

        if 'loc' in data:
            latitude, longitude = map(float, data['loc'].split(','))
            latitude+=28
            longitude+=7
            longitude*=-1
            return latitude, longitude
        else:
            return None
    except Exception as e:
        print()
        return None

# Example usage:
result = get_lat_long_by_ip()

if result:
    latitude, longitude = result
    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print("Could not retrieve location information.")


Latitude: 41.0878, Longitude: -87.2785


In [4]:
import math

def convert_lat_lon_to_xy(latitude, longitude):
    # Radius of the Earth in meters
    R = 6371000.0
    
    # Convert latitude and longitude to radians
    lat_rad = math.radians(latitude)
    lon_rad = math.radians(longitude)
    
    #y Calculate x and y coordinates using Mercator projection
    x = R * lon_rad
    y = R * math.log(math.tan(math.pi / 4 + lat_rad / 2))
    
    return x, y
x,y=convert_lat_lon_to_xy(41.0878,-87.2785)
x*=-1
x-=8564867
y-=3164867
print(x,y)

1140059.4051471204 1854810.5772076007


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report

df=pd.read_csv('Updated_dataset_Crime1.csv')
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Latitude,Longitude,Location,Alert,Month,Day,Hour,Minute,Second,emotion
0,12045583,JD226426,2020-05-07 10:24:00,12982,292,THEFT,2,17,False,False,...,41.830482,-87.621752,"(41.830481843, -87.621751752)",Danger,5,7,10,24,0,6
1,12031001,JD209965,2020-04-16 05:00:00,2355,222,BATTERY,2,17,True,False,...,41.836310,-87.639624,"(41.836310224, -87.639624112)",Safe,4,16,5,0,0,3
2,12093529,JD282112,2020-07-01 10:16:00,23418,6,ASSAULT,0,147,True,False,...,41.747610,-87.549179,"(41.747609555, -87.549179329)",Safe,7,1,10,16,0,0
3,12178140,JD381597,2020-09-27 23:29:00,21138,222,BATTERY,2,125,False,False,...,41.774878,-87.671375,"(41.77487752, -87.671374872)",Danger,9,27,23,29,0,0
4,12126129,JD321064,2020-08-04 20:28:00,23470,59,WEAPONS VIOLATION,0,147,True,False,...,41.746221,-87.658477,"(41.746220584, -87.658477307)",Safe,8,4,20,28,0,3


In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report

df=pd.read_csv('Updated_dataset_Crime1.csv')
# Assuming df is your DataFrame containing the dataset

# Splitting the data into features (X) and target variable (y)
X = df[['Description','emotion','Latitude','Longitude','X Coordinate','Y Coordinate','Year','Month','Day','Hour','Minute','Second']]
y = df['Alert']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
oversampler = RandomOverSampler(random_state=42)
X_train, y_train = oversampler.fit_resample(X_train, y_train)
# Creating and training the RandomForestClassifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicting on the testing set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
# Printing the classification report
print(classification_report(y_test, y_pred))

# Print the confusion matrix if needed
# from sklearn.metrics import confusion_matrix
#print(confusion_matrix(y_test, y_pred)

Accuracy: 0.8444605533405823
              precision    recall  f1-score   support

      Danger       0.87      0.95      0.91     34715
        Safe       0.53      0.28      0.36      6670

    accuracy                           0.84     41385
   macro avg       0.70      0.61      0.64     41385
weighted avg       0.82      0.84      0.82     41385



In [6]:
import joblib
#joblib.dump(model, 'RF_model.pkl')

In [7]:
loaded_model = joblib.load("C:/Users/Gnana Bharathi S/Downloads/Dec 14/RF1_81.pkl")
new_data=np.array([[0,3,41.840482,-87.621752,1178180.0,1881621.0,2020,10,20,12,23,0]])
# Standardize the new data using the same scaler
#new_data_standardized = scaler.transform(new_data)

# Make predictions for the new data
new_data_prediction = loaded_model.predict(new_data)
new_data_prediction[0]

'Safe'

In [65]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report

df=pd.read_csv('Updated_dataset_Crime1.csv')
# Assuming df is your DataFrame containing the dataset

# Splitting the data into features (X) and target variable (y)
X = df[['Description','emotion','Latitude','Longitude','X Coordinate','Y Coordinate','Year','Month','Day','Hour','Minute','Second']]
y = df['Alert']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
oversampler = RandomOverSampler(random_state=42)
X_train, y_train = oversampler.fit_resample(X_train, y_train)
y_pred = loaded_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
# Printing the classification report
print(classification_report(y_test, y_pred))

# Print the confusion matrix if needed
# from sklearn.metrics import confusion_matrix
#print(confusion_matrix(y_test, y_pred)

Accuracy: 0.9644073939833273
              precision    recall  f1-score   support

      Danger       0.97      0.99      0.98     34715
        Safe       0.94      0.84      0.88      6670

    accuracy                           0.96     41385
   macro avg       0.95      0.91      0.93     41385
weighted avg       0.96      0.96      0.96     41385



In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report

df=pd.read_csv('Updated_dataset_Crime1.csv')
# Assuming df is your DataFrame containing the dataset

# Splitting the data into features (X) and target variable (y)
X = df[['Description','emotion','Latitude','Longitude','X Coordinate','Y Coordinate','Year','Month','Day','Hour','Minute','Second']]
y = df['Alert']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
oversampler = RandomOverSampler(random_state=42)
X_train, y_train = oversampler.fit_resample(X_train, y_train)
y_pred = loaded_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
# Printing the classification report
print(classification_report(y_test, y_pred))

# Print the confusion matrix if needed
# from sklearn.metrics import confusion_matrix
#print(confusion_matrix(y_test, y_pred)

Accuracy: 0.9644073939833273
              precision    recall  f1-score   support

      Danger       0.97      0.99      0.98     34715
        Safe       0.94      0.84      0.88      6670

    accuracy                           0.96     41385
   macro avg       0.95      0.91      0.93     41385
weighted avg       0.96      0.96      0.96     41385



In [9]:
import numpy as np
#new_data = np.array([[0,0,41.747610,-87.549179,1198234.0,1851595.0,2020,7,1,10,16,0]])
new_data=np.array([[0,3,41.840482,-87.621752,1178180.0,1881621.0,2020,10,20,12,23,0]])
# Standardize the new data using the same scaler
#new_data_standardized = scaler.transform(new_data)

# Make predictions for the new data
new_data_prediction = loaded_model.predict(new_data)
print(f'Predicted Alert for the new data: {new_data_prediction[0]}')

Predicted Alert for the new data: Safe


In [27]:
new_data_prediction = loaded_model.predict(new_data)
print(f'Predicted Alert for the new data: {new_data_prediction[0]}')

Predicted Alert for the new data: Safe


In [19]:
from datetime import datetime
current_time = datetime.now()
hours = current_time.hour
minutes = current_time.minute
seconds = current_time.second
day=month=current_time.day
month=current_time.month
year=current_time.year

# Print the extracted values
print(f"Current Time: {current_time}")
print(f"Hours: {hours}")
print(f"Minutes: {minutes}")
print(f"Seconds: {seconds}")
print(f"Day: {day}")
print(f"Month: {month}")
print(f"Year: {year}")

result = get_lat_long_by_ip()
latitude, longitude = result
print('latitude = ',latitude)
print('longitude = ',longitude)
x, y = convert_lat_lon_to_xy(latitude, longitude)
print(f"X Coordinate: {x}, Y Coordinate: {y}")

# Replace these values with your own data
#new_data = np.array([[0,0,latitude, longitude, x, y, year, month, day, hours, minutes,seconds]])
#new_data = np.array([[1,2,41.830482, -87.621752, 1182280, 1881621, 2023, 12, 15, 14, 30,0]])
new_data=np.array([[0,1,41.747610,-87.549179,1198234.0,1851595.0,2020,7,1,10,16,0]])
# Standardize the new data using the same scaler
#new_data_standardized = scaler.transform(new_data)

# Make predictions for the new data
new_data_prediction = loaded_model.predict(new_data)

print(f'Predicted Alert for the new data: {new_data_prediction[0]}')

Current Time: 2024-02-24 20:42:36.647970
Hours: 20
Minutes: 42
Seconds: 36
Day: 24
Month: 2
Year: 2024
latitude =  13.0878
longitude =  80.2785
X Coordinate: 8926561.918635208, Y Coordinate: 1468120.358491257
Predicted Alert for the new data: Safe
